<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>

# Using a Benchmark Dataset to Build a Custom LLM as a Judge Evaluator

In this tutorial, you’ll learn how to build a custom LLM-as-a-Judge Evaluator tailored to your specific use case. While Phoenix provides several [pre-built evaluators](https://arize.com/docs/phoenix/evaluation/how-to-evals/running-pre-tested-evals) that have been tested against benchmark datasets, these may not always cover the nuances of your application.

So how can you achieve the same level of rigor when your use case falls outside the scope of standard evaluators?

We’ll walk through how to create your own benchmark dataset using a small set of annotated examples. This dataset will allow you to build and refine a custom evaluator by revealing failure cases and guiding iteration.

To follow along, you’ll need:

*   A free [Phoenix Cloud](https://app.arize.com/auth/phoenix/login) account
*   An OpenAI API Key



In [ ]:
!pip install -qqqq arize-phoenix arize-phoenix-otel openinference-instrumentation-openai

In [ ]:
!pip install -qq openai nest_asyncio

In [ ]:
import os

import nest_asyncio

nest_asyncio.apply()

from getpass import getpass

if not (phoenix_endpoint := os.getenv("PHOENIX_COLLECTOR_ENDPOINT")):
    os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = getpass("🔑 Enter your Phoenix Collector Endpoint: ")

if not (phoenix_api_key := os.getenv("PHOENIX_API_KEY")):
    os.environ["PHOENIX_API_KEY"] = getpass("🔑 Enter your Phoenix API key: ")

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    os.environ["OPENAI_API_KEY"] = getpass("🔑 Enter your OpenAI API key: ")

In [ ]:
from phoenix.otel import register

# configure the Phoenix tracer
tracer_provider = register(
    project_name="receipt-classification",  # Default is 'default'
    auto_instrument=True,
)

# Generate Image Classification Traces

In this tutorial, we’ll ask an LLM to generate expense reports from receipt images provided as public URLs. Running the cells below will generate traces, which you can explore directly in Phoenix for annotation. We’ll use GPT-4.1, which supports image inputs.


Dataset Information:
Jakob (2024). Receipt or Invoice Dataset. Roboflow Universe. CC BY 4.0. Available at: https://universe.roboflow.com/jakob-awn1e/receipt-or-invoice (accessed on 2025‑07‑29)

In [ ]:
import pandas as pd

urls = [
    "https://source.roboflow.com/Zf1kEIcRTrhHBZ7wgJleS4E92P23/8M5px2yLoNtZ6gOQ2r1D/original.jpg",
    "https://source.roboflow.com/Zf1kEIcRTrhHBZ7wgJleS4E92P23/8EVgYMNObyV6kLqBNeFG/original.jpg",
    "https://source.roboflow.com/Zf1kEIcRTrhHBZ7wgJleS4E92P23/86aohWmcEfO0XkflO8AB/original.jpg",
    "https://source.roboflow.com/HahhKcbQqdf8YAudM4kU3PuVCS72/1eGPBChz7wvovQROk2l8/original.jpg",
    "https://source.roboflow.com/HahhKcbQqdf8YAudM4kU3PuVCS72/0WqR2GSfGmxWB7ozo3Pj/original.jpg",
    "https://source.roboflow.com/HahhKcbQqdf8YAudM4kU3PuVCS72/FAEJRtviIboCYSKFZcEZ/original.jpg",
    "https://source.roboflow.com/HahhKcbQqdf8YAudM4kU3PuVCS72/0AoEaFy8FAw6DVieWCa8/original.jpg",
    "https://source.roboflow.com/HahhKcbQqdf8YAudM4kU3PuVCS72/0Q3hAyNwXNpHTeoWU7fz/original.jpg",
    "https://source.roboflow.com/HahhKcbQqdf8YAudM4kU3PuVCS72/2r876u4WpaCYFdMPwieK/original.jpg",
    "https://source.roboflow.com/HahhKcbQqdf8YAudM4kU3PuVCS72/2ZWeE0yO0oJUDtpgEAPY/original.jpg",
    "https://source.roboflow.com/HahhKcbQqdf8YAudM4kU3PuVCS72/37PF6xfHyuqzIBdO7Kgw/original.jpg",
    "https://source.roboflow.com/Zf1kEIcRTrhHBZ7wgJleS4E92P23/6mo4M0nJeKZEsdKrRfsR/original.jpg",
    "https://source.roboflow.com/Zf1kEIcRTrhHBZ7wgJleS4E92P23/5ezJ8tUBGbNnt0jZi2JU/original.jpg",
    "https://source.roboflow.com/Zf1kEIcRTrhHBZ7wgJleS4E92P23/4BCIWGazhCj03oTMWboO/original.jpg",
    "https://source.roboflow.com/Zf1kEIcRTrhHBZ7wgJleS4E92P23/4B8vXJNwJ7ZuHEWyjgAv/original.jpg",
    "https://source.roboflow.com/Zf1kEIcRTrhHBZ7wgJleS4E92P23/2EpeKbAqsSwciH2IHGyV/original.jpg",
    "https://source.roboflow.com/Zf1kEIcRTrhHBZ7wgJleS4E92P23/2LP3g9rKZrYDkNB3I78c/original.jpg",
    "https://source.roboflow.com/Zf1kEIcRTrhHBZ7wgJleS4E92P23/1hT6iLEIAFBw8W70u2FY/original.jpg",
    "https://source.roboflow.com/Zf1kEIcRTrhHBZ7wgJleS4E92P23/1zaKpaDhRPxkiIDTvMuc/original.jpg",
    "https://source.roboflow.com/Zf1kEIcRTrhHBZ7wgJleS4E92P23/1hF1R2Pt41hnlqhlXLDD/original.jpg",
]

In [ ]:
from openai import OpenAI

client = OpenAI()


def extract_receipt_data(input):
    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Analyze this receipt and return a brief summary for an expense report. Only include category of expense, total cost, and summary of items",
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": input,
                        },
                    },
                ],
            }
        ],
        max_tokens=500,
    )
    return response

In [ ]:
for url in urls:
    extract_receipt_data(url)

# Create Benchmarked Dataset

After generating traces, open Phoenix to begin annotating your dataset. In this example, we’ll annotate based on "accuracy", but you can choose any evaluation criterion that fits your use case. Just be sure to update the query below to match the annotation key you’re using—this ensures the annotated examples are included in your benchmark dataset.

![Annotated Data](https://storage.googleapis.com/arize-phoenix-assets/assets/images/build-your-first-eval-tutorial-1.png)

In [ ]:
import phoenix as px
from phoenix.client import Client
from phoenix.client.types import spans

client = Client(api_key=os.getenv("PHOENIX_API_KEY"))
# replace "accuracy" if you chose to annotate on different criteria
query = spans.SpanQuery().where("annotations['accuracy']")
spans_df = client.spans.get_spans_dataframe(
    query=query, project_identifier="receipt-classification"
)
annotations_df = client.spans.get_span_annotations_dataframe(
    spans_dataframe=spans_df, project_identifier="receipt-classification"
)
full_df = annotations_df.join(spans_df, how="inner")

dataset = px.Client().upload_dataset(
    dataframe=full_df,
    dataset_name="annotated-receipts",
    input_keys=["attributes.input.value"],
    output_keys=["attributes.llm.output_messages"],
    metadata_keys=["result.label", "result.score", "result.explanation"],
)

# Create evaluation template

Next, we’ll create a baseline evaluation template and define both the task and the evaluation function. Once these are set up, we’ll run an experiment to compare the evaluator’s performance against our ground truth annotations.

In [ ]:
from phoenix.evals.templates import (
    ClassificationTemplate,
    PromptPartContentType,
    PromptPartTemplate,
)

rails = ["accurate", "almost accurate", "inaccurate"]
classification_template = ClassificationTemplate(
    rails=rails,  # Specify the valid output labels
    template=[
        # Prompt part 1: Task description
        PromptPartTemplate(
            content_type=PromptPartContentType.TEXT,
            template=""" You are an evaluator tasked with assessing the quality of a model-generated expense report based on a receipt.
Below is the model’s generated expense report and the input image:
---
MODEL OUTPUT (Expense Report): {output}

---
INPUT RECEIPT: """,
        ),
        # Prompt part 2: Insert the image data
        PromptPartTemplate(
            content_type=PromptPartContentType.IMAGE,
            template="{image}",  # Placeholder for the image URL
        ),
        # Prompt part 3: Define the response format
        PromptPartTemplate(
            content_type=PromptPartContentType.TEXT,
            template=""" Evaluate the following three aspects and assign one of the following labels for each. Only include the label:
- **"accurate"** – Fully correct
- **"almost accurate"** – Mostly correct
- **"inaccurate"** – Substantially wrong
            """,
        ),
    ],
)

print(classification_template)

In [ ]:
import json

from phoenix.evals import OpenAIModel, llm_classify


def task_function(input, reference):
    parsed = json.loads(input["attributes.input.value"])
    image_url = parsed["messages"][0]["content"][1]["image_url"]["url"]
    output = reference["attributes.llm.output_messages"][0]["message.content"]

    response_classification = llm_classify(
        data=pd.DataFrame([{"image": image_url, "output": output}]),
        template=classification_template,
        model=OpenAIModel(model="gpt-4o"),
        rails=rails,
        provide_explanation=True,
    )
    # print(response_classification)
    label = response_classification.iloc[0]["label"]
    return label


def evaluate_response(output, metadata):
    expected_label = metadata["result.label"]
    predicted_label = output
    return 1 if expected_label == predicted_label else 0

In [ ]:
from phoenix.experiments import run_experiment

dataset = px.Client().get_dataset(name="annotated-receipts")

initial_experiment = run_experiment(
    dataset,
    task=task_function,
    evaluators=[evaluate_response],
    experiment_name="initial template",
)

#Iteration 1 to improve evaluator prompt template

Next, we’ll refine our evaluation prompt template by adding more specific instructions to classification rules. We can add these rules based on gaps we saw in the previous iteration. This additional guidance helps improve accuracy and ensures the evaluator's judgments better align with human expectations.

In [ ]:
classification_template = ClassificationTemplate(
    rails=rails,  # Specify the valid output labels
    template=[
        # Prompt part 1: Task description
        PromptPartTemplate(
            content_type=PromptPartContentType.TEXT,
            template=""" You are an evaluator tasked with assessing the quality of a model-generated expense report based on a receipt.
Below is the model’s generated expense report and the input image:
---
MODEL OUTPUT (Expense Report): {output}

---
INPUT RECEIPT: """,
        ),
        # Prompt part 2: Insert the audio data
        PromptPartTemplate(
            content_type=PromptPartContentType.IMAGE,
            template="{image}",  # Placeholder for the image URL
        ),
        # Prompt part 3: Define the response format
        PromptPartTemplate(
            content_type=PromptPartContentType.TEXT,
            template=""" Evaluate the following and assign one of the following labels for each. Only include the label:
- **"accurate"** – Total price, itemized list, and expense category are all accurate. All three must be correct to get this label.
- **"almost accurate"** – Mostly correct but with small issues. For example, expense category is too vague.
- **"inaccurate"** – Substantially wrong or missing information. For example, incorrect total price.
            """,
        ),
    ],
)

In [ ]:
initial_experiment = run_experiment(
    dataset,
    task=task_function,
    evaluators=[evaluate_response],
    experiment_name="improved template",
)

#Iteration 2 to improve evaluator prompt template

To further improve our evaluator, we’ll introduce few-shot examples into the evaluation prompt. These examples help highlight common failure cases and guide the evaluator toward more consistent and generalized judgments.

In [ ]:
classification_template = ClassificationTemplate(
    rails=rails,  # Specify the valid output labels
    template=[
        # Prompt part 1: Task description
        PromptPartTemplate(
            content_type=PromptPartContentType.TEXT,
            template=""" You are an evaluator tasked with assessing the quality of a model-generated expense report based on a receipt.
Below is the model’s generated expense report and the input image:
---
MODEL OUTPUT (Expense Report): {output}

---
INPUT RECEIPT: """,
        ),
        # Prompt part 2: Insert the audio data
        PromptPartTemplate(
            content_type=PromptPartContentType.IMAGE,
            template="{image}",  # Placeholder for the image URL
        ),
        # Prompt part 3: Define the response format
        PromptPartTemplate(
            content_type=PromptPartContentType.TEXT,
            template=""" Evaluate the following three aspects and assign one of the following labels for each. Only include the label:
- **"accurate"** – Total price, itemized list, and expense category are accurate. All three must be correct to get this label.
  An incorrect category is one that is overly vague (e.g., “Miscellaneous”, "Supplies") or does not accurately reflect the itemized list.
  For example, "Dining and Entertainment" should not be grouped together if the itemized list only includes food.
  Reasonable general categories like “Office Supplies” or “Groceries” are acceptable if they align with the listed items.

- **"almost accurate"** – Mostly correct but with small issues. For example, expense category is too vague.
  If a category includes extra fields (ex: "Dining and Entertainment", but the receipt only includes food) mark this as almost correct.
- **"inaccurate"** – Substantially wrong or missing. For example, incorrect total price or one of more of the items is missing makes the total result inaccurate.
            """,
        ),
    ],
)

In [ ]:
initial_experiment = run_experiment(
    dataset,
    task=task_function,
    evaluators=[evaluate_response],
    experiment_name="few shot improved template",
)

# Final Results

Once your evaluator reaches a performance level you're satisfied with, it's ready for use. The target score will depend on your benchmark dataset and specific use case. That said, you can continue applying the techniques from this tutorial to refine and iterate until the evaluator meets your desired level of quality.

![Final Results](https://storage.googleapis.com/arize-phoenix-assets/assets/images/build-your-first-eval-tutorial-2.png)